**ML Final Project**
- John Blythe and John Mekus
- Predicting gene class based on gene sequence. Done through training an ML linear regression model, then moving to a neural network model.

To use pydna library go into terminal and enter: pip install pydna

In [ ]:
# RUN THIS
# creating pandas dataframe of gene sequence/class data from .txt file
import pydna
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

gene_df = pd.read_csv('human_data.txt', sep="\t", header=None)
gene_df.columns = ['seq','class']


In [ ]:
gene_df

**Training Linear Regression Model**

In [ ]:
# DON'T RUN
# Attemped binary one hot encoding, still experiencing problems with variable length of x data - doesn't work
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
x_train,x_test,y_train,y_test = train_test_split(gene_df['seq'],gene_df['class'])

x_train_encoded = []
for i in x_train:
    seq_list = np.array(list(i))
    encoder = LabelEncoder()
    onehot = OneHotEncoder()
    int_encode = encoder.fit_transform(seq_list)
    int_encode = int_encode.reshape(len(int_encode),1)
    onehot_encoded = onehot.fit_transform(int_encode)
    onehot_encoded.reshape(1,-1)
    x_train_encoded.append(onehot_encoded)

x_train_encoded_clean = []
for i in x_train_encoded:
    if np.shape(i)[0]%2 != 0:
        mod = i[:np.shape(i)[0]-1]
        x_train_encoded_clean.append(mod)
    else:
        x_train_encoded_clean.append(i)
        
x_train_encoded_clean = np.array(x_train_encoded_clean)
model = LinearRegression()
model.fit(x_train_encoded_clean.reshape(-1,1),y_train)
        
        
        


In [ ]:
# RUN THIS
# batching data of the same length and training/testing on Linear Regression model - works
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error as mse


x_train,x_test,y_train,y_test = train_test_split(gene_df['seq'],gene_df['class'])
length_1038 = []
for i in x_train:
    if len(i) == 1038:
        length_1038.append(i)
x_train_encoded = []
for i in length_1038:
    seq_list = np.array(list(i))
    encoder = LabelEncoder()
    int_encode = encoder.fit_transform(seq_list)
    x_train_encoded.append(int_encode)

x_train_encoded = np.array(x_train_encoded)

test_1038 = []
for i in x_test:
    if len(i) == 1038:
        test_1038.append(i)
test_encoded = []
for i in test_1038:
    seq_list = np.array(list(i))
    encoder = LabelEncoder()
    int_encode = encoder.fit_transform(seq_list)
    test_encoded.append(int_encode)

model = LinearRegression()
lr = model.fit(x_train_encoded,y_train[0:len(x_train_encoded)])
error = mse(model.predict(test_encoded),y_test[0:len(test_encoded)])
print(error)
#clf.predict_proba(X_test[:1])
#clf.predict(X_test)


In [ ]:
# DON'T RUN THIS
# using standard scaler on dataset - doesn't work
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

x_train,x_test,y_train,y_test = train_test_split(gene_df['seq'],gene_df['class'])
scaler = StandardScaler()
x_train_encoded = []
for i in x_train:
    seq_list = np.array(list(i))
    encoder = LabelEncoder()
    int_encode = encoder.fit_transform(seq_list)
    x_train_encoded.append(int_encode)

x_train_encoded = np.array(x_train_encoded)
x_train_scaled = scaler.fit(x_train_encoded)
x_train_scaled = scaler.transfor(x_train_scaled)

test_encoded = []
for i in x_test:
    seq_list = np.array(list(i))
    encoder = LabelEncoder()
    int_encode = encoder.fit_transform(seq_list)
    test_encoded.append(int_encode)

test_encoded = np.array(test_encoded)
test_scaled = scaler.fit(test_encoded)
test_scaled = scaler.transfor(test_scaled)

model = LogisticRegression()
lr = model.fit(x_train_scaled,y_train[0:len(x_train_scaled)])
error = mse(model.predict(test_scaled),y_test[0:len(test_scaled)])
print(error)
#clf.predict_proba(X_test[:1])
#clf.predict(X_test)




In [ ]:
# prints dictionary of length frequencies in dataset
# used to determine sequence lengths to use when manually batching dataset into arrays of the same length
# for linear regression model
import collections
y = collections.Counter()
x_train,x_test,y_train,y_test = train_test_split(gene_df['seq'],gene_df['class'])
same_length = []
for i in x_test:
    same_length.append(len(i))
    
y = collections.Counter(same_length)
print(y)



**Implementing Kernel Method and SVC Multiclass Classifier**

In [ ]:
# RUN THIS
# SVC multiclassifier without encoding using string kernel
# Ran a for loop that increased polynomial degree on each iteration, showed no change in mse, so it was not included
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import svm
from stringkernels.kernels import string_kernel
from sklearn.metrics import mean_squared_error as mse

x_train,x_test,y_train,y_test = train_test_split(gene_df['seq'],gene_df['class'])
svm_sk_model = svm.SVC(gamma = 'auto',degree=2, kernel=string_kernel())
svm_sk_model.fit(x_train, y_train)
error = mse(svm_sk_model.predict(x_test),y_test)
print(error)
